In [ ]:
import os
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cv2
from PIL import Image
#import imageio, skimage

import torch

from collections import Counter

In [ ]:
%pwd

In [ ]:
print("GPU available:", torch.cuda.is_available())
print("Device count :", torch.cuda.device_count())

In [ ]:
%load_ext autoreload
%autoreload 2

%reload_ext autoreload

In [ ]:
from virtex.data.datasets.arch_captions import ArchCaptionsDatasetRaw

## Unified Dataset Class for ARCH Dataset

In [ ]:
arch_dataset_raw_train = ArchCaptionsDatasetRaw(data_root='../datasets/ARCH',
                                                source="both",
                                                split="train")
len(arch_dataset_raw_train.instances)

## Unified Dataset Class + augmentations and collate function

In [ ]:
import random
from typing import Callable, Dict, List

import albumentations as alb
import numpy as np
import torch
from torch.utils.data import Dataset

from virtex.data.tokenizers import SentencePieceBPETokenizer
from virtex.data import transforms as T
from virtex.data.datasets.arch_captions import ArchCaptionsDatasetRaw


class ArchCaptioningDatasetExtended(Dataset):
    r"""
    A dataset which provides image-caption (forward and backward) pairs from
    a ARCH Captions annotation file. This is used for pretraining tasks which
    use captions - bicaptioning, forward captioning and token classification.

    Args:
        data_root: Path to dataset directory containing images and annotations.
        source: Name of ARCH source to read. One of ``{"pubmed", "books", "both"}``.
            "both" option results in a concatenation of the datasets from "pubmed" and "books"
        split: Name of ARCH split to read. One of ``{"train", "val", "all"}``.
        tokenizer: Tokenizer which maps word tokens to their integer IDs.
        image_transform: List of image transformations, from either
            `albumentations <https://albumentations.readthedocs.io/en/latest/>`_
            or :mod:`virtex.data.transforms`.
        max_caption_length: Maximum number of tokens to keep in caption tokens.
            Extra tokens will be trimmed from the right end of the token list.
    """

    def __init__(
            self,
            data_root: str,
            split: str,
            tokenizer: SentencePieceBPETokenizer,
            source: str = "both",
            image_transform: Callable = T.ARCH_DEFAULT_IMAGE_TRANSFORM,
            tensor_flip_transform: Callable = None,
            max_caption_length: int = 30,
    ):
        self._dset = ArchCaptionsDatasetRaw(data_root=data_root, source=source,
                                            split=split)
        self.image_transform = image_transform
        self.tensor_flip_transform = tensor_flip_transform
        self.caption_transform = alb.Compose(
            [
                T.NormalizeCaption(),
                T.TokenizeCaption(tokenizer),
                T.TruncateCaptionTokens(max_caption_length),
            ]
        )
        self.padding_idx = tokenizer.token_to_id("<unk>")

    def __len__(self):
        return len(self._dset)

    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        # keys: {"image_ids", "images", "caption"}
        instance = self._dset[idx]
        image_ids, images, caption = (
            instance["image_ids"],
            instance["images"],
            instance["caption"],
        )

        # # debugging
        # print("Checkpoint 1")
        # print("Shapes before applying self.image_transform", [image.shape for image in images])

        # List[int] -> np.array of shape (len(image_ids), )
        image_ids = np.array(image_ids)
        # (len(image_ids), ) -> (len(image_ids), 1)
        image_ids = image_ids.reshape((image_ids.shape[0], 1))

        # # debugging
        # print("Checkpoint 2")

        # Transform images, no flips at this stage not to create multiple versions of the caption!
        #     Before flipping all images need to be resized to the same size to put them into a tensor.
        #     Caption won't be tokenized/processed here.
        #     Albumentations transforms require named arguments - can't avoid it.

        images = [self.image_transform(image=image)["image"] for image in
                  images]
        # print("Shapes after applying self.image_transform", [image.shape for image in images])

        # # # debugging
        # print("Checkpoint 3")

        # Convert each image from HWC to CHW format and convert to tensors:
        #     PyTorch Transforms expect to receive tensors in (B, C, H, W) shape
        #     [(Channel, Height, Width), ..., ] Bag Size times
        images = [np.transpose(image, (2, 0, 1)) for image in images]
        images = [torch.tensor(image, dtype=torch.float) for image in images]

        # # # debugging
        # print("Checkpoint 4")

        # stack all the images into a tensor: (bag_size=batch_size, Channel, Height, Width)
        images = torch.stack(images, dim=0)

        if self.tensor_flip_transform is not None:
            # perform tensor transforms on images in the tensor and the
            # corresponding caption, e.g. random horizontal flips
            # Reason: single version of the caption should appear => random flip
            # should be performed on all images in a bag
            images_caption = self.tensor_flip_transform(image=images, caption=caption)
            images, caption = images_caption["image"], images_caption["caption"]

        # print(images)
        # print(caption)

        # # # debugging
        # print("Checkpoint 5")

        # caption tokens
        caption_tokens = self.caption_transform(caption=caption)["caption"]

        # # # debugging
        # print("Checkpoint 6")

        return {
            "image_ids": torch.tensor(image_ids, dtype=torch.long), #(bag_size,1)
            "images": images,
            "caption_tokens": torch.tensor(caption_tokens, dtype=torch.long),
            "noitpac_tokens": torch.tensor(caption_tokens,
                                           dtype=torch.long).flip(0),
            "caption_lengths": torch.tensor(len(caption_tokens),
                                            dtype=torch.long),
        }

    def collate_fn(
            self, data: List[Dict[str, torch.Tensor]]
    ) -> Dict[str, torch.Tensor]:
        # Pad `caption_tokens` and `masked_labels` up to this length.
        caption_tokens = torch.nn.utils.rnn.pad_sequence(
            [d["caption_tokens"] for d in data],
            batch_first=True,
            padding_value=self.padding_idx,
        )
        noitpac_tokens = torch.nn.utils.rnn.pad_sequence(
            [d["noitpac_tokens"] for d in data],
            batch_first=True,
            padding_value=self.padding_idx,
        )
        return {
            "image_id": torch.stack([d["image_ids"] for d in data], dim=0),
            "image": torch.stack([d["images"] for d in data], dim=0),
            "caption_tokens": caption_tokens,
            "noitpac_tokens": noitpac_tokens,
            "caption_lengths": torch.stack(
                [d["caption_lengths"] for d in data]),
        }


In [ ]:
# check the default transform
T.DEFAULT_IMAGE_TRANSFORM

In [ ]:
T.ARCH_DEFAULT_IMAGE_TRANSFORM

In [ ]:
help(T.DEFAULT_FLIP_TRANSFORM)

In [ ]:
#help(SentencePieceBPETokenizer)

In [ ]:
arch_tokenizer = SentencePieceBPETokenizer("../datasets/vocab/arch_10k.model")

arch_train_dataset_extended = ArchCaptioningDatasetExtended(data_root='../datasets/ARCH',
                                                      split="train",
                                                      tokenizer=arch_tokenizer,
                                                      tensor_flip_transform=None)
len(arch_train_dataset_extended)

In [ ]:
sample_item = arch_train_dataset_extended.__getitem__(0)
sample_item

In [ ]:
sample_item['images'].size()

In [ ]:
sample_tensor_image = sample_item['images'][0]
sample_tensor_image

In [ ]:
plt.imshow(sample_tensor_image.to(int).view(sample_tensor_image.shape[1],
                                     sample_tensor_image.shape[2], sample_tensor_image.shape[0]))

### Test how the dataloader works

This will help to debug the `collate_fn`

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
        dataset=arch_train_dataset_extended,
        batch_size=2,
        shuffle=True,
        drop_last=True,
        collate_fn=arch_train_dataset_extended.collate_fn,
    )

len(train_dataloader)


In [ ]:
for batch in train_dataloader:
    print("image_id", batch["image_id"])
    print("image shape", batch["image"].shape)
    print("caption_tokens:", batch["caption_tokens"])
    print("noitpac_tokens:", batch["noitpac_tokens"])
    print("caption_lengths:", batch["caption_lengths"])
    break

In [ ]:
batch = next(iter(train_dataloader))
batch

In [ ]:
batch['image'].shape

### Examples of TensorHorizontalFlip

In [ ]:
flip = T.TensorHorizontalFlip(p=0.5)
img_tensor = torch.Tensor([[[1, 1, 0, 0] for _ in range(4)],
                           [[0, 0, 1, 1] for _ in range(4)]])
print(img_tensor.shape)
img_tensor

In [ ]:
image_caption = flip(image=img_tensor, caption="1-s are to the left on the "
                                               "first image and to the right "
                                               "on the second")
print(image_caption['image'])
print(image_caption['caption'])

### Checking if the horizontal flip can be applied to different images separately in torchvision

In [ ]:
import torchvision
from PIL import Image
test_transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToPILImage(),
    torchvision.transforms.RandomHorizontalFlip(p=0.5),
    torchvision.transforms.ToTensor()]
)

fixed_test_transform = test_transform

In [ ]:
test_list = [np.array([1, 1, 0, 0], dtype=np.float32).reshape((1, 4)) for _ in range(20)]
test_list

In [ ]:
[fixed_test_transform(l) for l in test_list]

It can not be applied separately, so the flip needs to be applied on the tensor